In [29]:
import os
import cv2
import open3d as o3d
import torch
import json
import yaml
import numpy as np
import rosbag
import matplotlib.pyplot as plt
import data_conversion
import depth_anything_interface
import pcd_utils

In [30]:
FRAME_INDEX = 0
GAP_INDEX = 10
MODEL_PATH = "/scratchdata/depth_anything_v2_metric_hypersim_vitl.pth"
model = depth_anything_interface.get_model("cuda", MODEL_PATH, model_type = "metric", encoder='vitl')

config = yaml.load(open("../config/gemini2L.yaml", "r"), Loader=yaml.FullLoader)

# Open bag file
bag_file_path = "/scratchdata/indoor_short.bag"
bag = rosbag.Bag(bag_file_path)

INFO - 2024-10-16 02:30:40,251 - dinov2 - using MLP layer as FFN


In [31]:
wait = 0
for topic, msg, t in bag.read_messages(topics=["/camera/color/camera_info"]):
    D = msg.D
    K = msg.K
    R = msg.R
    P = msg.P
    break

fx = P[0]
fy = P[5]
cx = P[2]
cy = P[6]

print(fx, fy, cx, cy)

306.4570007324219 306.4668884277344 319.01312255859375 197.51637268066406


In [32]:
cnt = 0 
for topic, msg, t in bag.read_messages(topics=["/camera/color/image_raw"]):
    if cnt ==  FRAME_INDEX:
        prev_img = data_conversion.topic_to_image(msg)
    if cnt == GAP_INDEX + FRAME_INDEX:
        new_img = data_conversion.topic_to_image(msg)
        break
    cnt+=1

cnt = 0 
for topic, msg, t in bag.read_messages(topics=["/camera/depth/image_raw"]):
    if cnt ==  FRAME_INDEX:
        prev_depth = data_conversion.topic_to_depth(msg, config["depth_anything_config"])
    if cnt == GAP_INDEX + FRAME_INDEX:
        new_depth = data_conversion.topic_to_depth(msg, config["depth_anything_config"])
        break
    cnt+=1


In [33]:
#prev_est_depth = prev_depth
#new_est_depth = new_depth * 0.9

prev_est_depth = model.infer_image(prev_img)
new_est_depth = model.infer_image(new_img)

In [34]:
gray_prev = cv2.cvtColor(prev_img, cv2.COLOR_BGR2GRAY)
gray_new = cv2.cvtColor(new_img, cv2.COLOR_BGR2GRAY)

flow = cv2.calcOpticalFlowFarneback(gray_prev, gray_new, None, 0.5, 3, 15, 3, 5, 1.2, 0)

prev_point = np.indices((prev_img.shape[0], prev_img.shape[1]))
prev_point = np.moveaxis(prev_point, 0, -1)

new_point = prev_point + flow

mask = np.linalg.norm(flow, axis=2) > 20 # Match based on nearer objects, these should be more accurate?
prev_point = prev_point[mask==1]
new_point = new_point[mask==1]

mask = new_point[:, 1] < new_img.shape[0] - 1 
prev_point = prev_point[mask]
new_point = new_point[mask]

mask = prev_point[:, 1] < new_img.shape[0] - 1
prev_point = prev_point[mask]
new_point = new_point[mask]

mask = new_point[:, 0] < new_img.shape[1] - 1 
prev_point = prev_point[mask]
new_point = new_point[mask]

mask = prev_point[:, 0] < new_img.shape[1] - 1
prev_point = prev_point[mask]
new_point = new_point[mask]

matching_depth1 = data_conversion.interpolate_depth(prev_est_depth, prev_point)
matching_depth2 = data_conversion.interpolate_depth(new_est_depth, new_point)

coord1 = np.zeros((len(matching_depth1), 3), dtype=np.float32)

coord1[:, 0] = (prev_point[:,0] - cx) * matching_depth1/ fx
coord1[:, 1] = (prev_point[:,1] - cy) * matching_depth1/ fy
coord1[:, 2] = matching_depth1

coord2 = np.zeros((len(matching_depth2), 3), dtype=np.float32)

coord2[:, 0] = (new_point[:,0] - cx) * matching_depth2/ fx
coord2[:, 1] = (new_point[:,1] - cy) * matching_depth2/ fy
coord2[:, 2] = matching_depth2

print(coord1.shape, coord2.shape)

(62933, 3) (62933, 3)


In [35]:
EPISON = 0.1
RANSAC_ITERATIONS = 50000

best_inliers = 0
best_tf = np.eye(4)

for _ in range(RANSAC_ITERATIONS):
    sample_indices = np.random.choice(len(coord1), 3, replace=False)
    sample1 = coord1[sample_indices]
    sample2 = coord2[sample_indices]

    # Zero Mean
    sample1_mean = np.mean(sample1, axis=0)
    sample2_mean = np.mean(sample2, axis=0)

    sample1 = sample1 - sample1_mean
    sample2 = sample2 - sample2_mean

    # SVD
    H = sample2.T @ sample1
    try:
        U, D, V_T = np.linalg.svd(H)
    except:
        continue
    
    R = U @ V_T

    sample2 = sample2 @ R.T

    scale = np.einsum('ij,ij->', sample1, sample2) / np.einsum('ij,ij->', sample2, sample2)
    #print(np.einsum('ij,ij->', sample1, sample2), np.einsum('ij,ij->', sample2, sample2))

    if scale < 1e-5:
        continue
        
    test_coord2 = (coord2 - sample2_mean) * scale
    test_coord2 = test_coord2 @ R.T + sample1_mean

    error = np.linalg.norm(test_coord2 - coord1, axis=1)

    inliers = np.mean(error < EPISON)

    if (inliers > best_inliers):
        best_inliers = inliers

        tf = np.eye(4)
        tf[:3,3] = -sample2_mean.T

        new_tf = np.eye(4)
        new_tf[3,3] = 1/scale
        tf = new_tf @ tf

        new_tf = np.eye(4)
        new_tf[:3,:3] = R
        tf = new_tf @ tf

        new_tf = np.eye(4)
        new_tf[:3,3] = sample1_mean.T
        tf = new_tf @ tf

        best_tf = tf

print(best_inliers)
print(best_tf)

/tmp/ipykernel_2363/3434762136.py:30: RuntimeWarning: invalid value encountered in scalar divide
  scale = np.einsum('ij,ij->', sample1, sample2) / np.einsum('ij,ij->', sample2, sample2)


0.5132601337930816
[[ 0.99656808  0.00106347 -0.08276974 -0.01713545]
 [-0.00585026  0.99832195 -0.05761163  0.02939722]
 [ 0.08256958  0.05789815  0.99490201  0.05235686]
 [ 0.          0.          0.          0.90407683]]


In [36]:
prev_coord = data_conversion.depth_to_pcd(prev_est_depth,P) 
#prev_coord = prev_coord - mean_ori
#prev_coord = prev_coord / scale_ori

prev_coord = np.concatenate([prev_coord, np.ones((prev_coord.shape[0], 1))], axis=1)

prev_pcd = o3d.geometry.PointCloud()
prev_pcd.points = o3d.utility.Vector3dVector(prev_coord[:,:3]/prev_coord[:,3][:,None])
prev_pcd.colors = o3d.utility.Vector3dVector(prev_img.reshape(-1,3)/255.0)

#prev_pcd = prev_pcd.uniform_down_sample(every_k_points=4)
prev_pcd.voxel_down_sample(0.1)


new_coord = data_conversion.depth_to_pcd(new_est_depth,P)
#new_coord = new_coord - mean_new
#new_coord = new_coord / scale_new

new_coord = np.concatenate([new_coord, np.ones((new_coord.shape[0], 1))], axis=1)
new_coord = new_coord @ best_tf.T

new_pcd = o3d.geometry.PointCloud()
new_pcd.points = o3d.utility.Vector3dVector(new_coord[:,:3]/new_coord[:,3][:,None])
new_pcd.colors = o3d.utility.Vector3dVector(new_img.reshape(-1,3)/255.0)

#new_pcd = new_pcd.uniform_down_sample(every_k_points=4)
new_pcd.voxel_down_sample(0.1)

PointCloud with 10172 points.

In [37]:
o3d.visualization.draw_geometries([prev_pcd, new_pcd])

In [38]:
voxel_radius = [0.04, 0.02, 0.01]
max_iter = [50, 30, 14]
current_transformation = np.identity(4)
print("3. Colored point cloud registration")
for scale in range(3):
    iter = max_iter[scale]
    radius = voxel_radius[scale]
    print([iter, radius, scale])

    print("3-1. Downsample with a voxel size %.2f" % radius)
    source_down = prev_pcd.voxel_down_sample(radius)
    target_down = new_pcd.voxel_down_sample(radius)

    print("3-2. Estimate normal.")
    source_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))
    target_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))

    print("3-3. Applying colored point cloud registration")
    result_icp = o3d.pipelines.registration.registration_colored_icp(
        source_down, target_down, radius, current_transformation,
        o3d.pipelines.registration.TransformationEstimationForColoredICP(),
        o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                          relative_rmse=1e-6,
                                                          max_iteration=iter))
    current_transformation = result_icp.transformation
    print(result_icp)

o3d.visualization.draw_geometries([prev_pcd, new_pcd])

3. Colored point cloud registration
[50, 0.04, 0]
3-1. Downsample with a voxel size 0.04
3-2. Estimate normal.
3-3. Applying colored point cloud registration
RegistrationResult with fitness=6.219243e-01, inlier_rmse=2.273578e-02, and correspondence_set size of 21383
Access transformation to get result.
[30, 0.02, 1]
3-1. Downsample with a voxel size 0.02
3-2. Estimate normal.
3-3. Applying colored point cloud registration
RegistrationResult with fitness=4.784641e-01, inlier_rmse=1.224865e-02, and correspondence_set size of 36447
Access transformation to get result.
[14, 0.01, 2]
3-1. Downsample with a voxel size 0.01
3-2. Estimate normal.
3-3. Applying colored point cloud registration
RegistrationResult with fitness=2.410612e-01, inlier_rmse=6.763303e-03, and correspondence_set size of 31485
Access transformation to get result.
